In [27]:
wsj0path = "/project/data_asr/wham_dataset/whamr_data/wsj0_raw/"

cv = []
with open('mix_2_spk_cv.txt','r') as txt:
    out = txt.readline()
    while out != "":
        out = out.split()
        cv.append(out)
        out = txt.readline()

tr = []
with open('mix_2_spk_tr.txt','r') as txt:
    out = txt.readline()
    while out != "":
        out = out.split()
        tr.append(out)
        out = txt.readline()

tt = []
with open('mix_2_spk_tt.txt','r') as txt:
    out = txt.readline()
    while out != "":
        out = out.split()
        tt.append(out)
        out = txt.readline()

print("tr:",len(tr))
print("cv:",len(cv))
print("tt:",len(tt))

tr: 20000
cv: 5000
tt: 3000


In [28]:
import torchaudio

a,b = torchaudio.load(wsj0path+tr[0][0])
a

tensor([[-3.0518e-05, -1.2207e-04,  1.2207e-04,  ..., -3.0518e-05,
         -9.1553e-05, -2.4414e-04]])